### Задача:

Прогноз продаж одной из популярных моделей [фичерфонов](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D1%87%D0%B5%D1%80%D1%84%D0%BE%D0%BD) (на картинке ниже пример похожего устройства) в салонах МегаФона
![](https://39.img.avito.st/640x480/8468720439.jpg)

### Исходные данные:

Датасет содержит следующие поля:

1. `point_id` - Индентификатор салона
2. `lon` - Долгота точки
3. `lat` - Широта точки
4. `target` - Значение таргета, усредненное за несколько месяцев и отнормированное

### Требования к решению и советы:

Ниже приведен список из нескольких важных пунктов, необходимых для решения задания. Выполнение каждого из пунктов влияет на итоговую оценку. Вы можете выполнить каждый из пунктов разными способами, самым лучшим будет считаться вариант, когда всё получение и обработка данных будут реализованы на Питоне (пример: вы можете скачать данные из OSM через интерфейс на сайте overpass-turbo или с помощью библиотек `overpass`/`requests`. Оба варианта будут зачтены, но больше баллов можно заработать во втором случае)



1. Салоны расположены в нескольких разных городах, вам необходимо **определить город для каждого салона** (это понадобится во многих частях задания). К этому есть разные подходы. Вы можете провести [обратное геокодирование](https://en.wikipedia.org/wiki/Reverse_geocoding) с помощью геокодера [Nominatim](https://nominatim.org/), доступного через библиотеку `geopy` примерно вот так:
```python
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)
```
В таком случае, вам придется обрабатывать полученную строку адреса, чтобы извлечь название города. Также вы можете скачать из OSM или найти в любом другом источнике границы административно территориальных границ России и пересечь с ними датасет с помощью `geopandas.sjoin` (этот вариант более надежный, но нужно будет разобраться с тем, как устроены границы АТД в OSM, обратите внимание на [этот тег](https://wiki.openstreetmap.org/wiki/Key:admin_level))


2. **Используйте данные OSM**: подумайте, какие объекты могут влиять на продажи фичерфонов. Гипотеза: такие телефоны покупают люди, приезжающие в город или страну ненадолго, чтобы вставить туда отдельную симкарту для роуминга. Можно попробовать использовать местоположения железнодорожных вокзалов (изучите [этот тег](https://wiki.openstreetmap.org/wiki/Tag:railway%3Dstation)). Необходимо использовать хотя бы 5 разных типов объектов из OSM. Скорее всего, вам придется качать данные OSM отдельно для разных городов (см. пример для Нью-Йорка из лекции)


3. **Используйте разные способы генерации признаков**: описать положение салона МегаФона относительно станций метро можно разными способами - найти ***расстояние до ближайшей станции***, или же посчитать, сколько станций попадает в ***500 метровую буферную зону*** вокруг салона. Такие признаки будут нести разную информацию. Так же попробуйте поэкспериментировать с размерами буферных зон (представьте, что значат в реальности радиусы 100, 500, 1000 метров). Попробуйте посчитать расстояние до центра города, до других объектов.

4. **Сделайте визуализации**: постройте 2-3 карты для какого нибудь из городов - как распределен в пространстве таргет, где находятся объекты, полученные вами из OSM. Можете использовать любой инструмент - обычный `plot()`, `folium`, `keplergl`. Если выберете Кеплер, обязательно сохраните в файл конфиг карты, чтобы ее можно было воспроизвести. Сделать это можно вот так:

```python
import json
json_data = kepler_map.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)
```
5. Задание не ограничено приведенными выше пунктами, попробуйте нагенерировать интересных признаков, найти в интернете дополнительные данные (в таком случае в комментарии к коду укажите ссылку на ресурс, откуда взяли данные)



6. Это довольно сложная задача - датасет очень маленький, данные по своей природе довольно случайны. Поэтому место и скор на Kaggle не будут играть решающую роль в оценке, но позволят заработать дополнительные баллы

In [ ]:
!pip install geopandas
!pip install overpass

     |████████████████████████████████| 972kB 4.9MB/s 
     |████████████████████████████████| 6.5MB 19.4MB/s 
     |████████████████████████████████| 14.8MB 306kB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import json

import pandas as pd
import numpy as np
import geopandas as gpd
import overpass
from geopy.geocoders import Nominatim
from geopy.location import Location
from shapely.geometry import Point
from geopy import distance

import folium
from folium.plugins import HeatMap

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/MyDrive/Colab Notebooks/geo-data'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Read data

In [ ]:
train = pd.read_csv('{}/mf_geo_train.csv'.format(DATA_PATH))
test = pd.read_csv('{}/mf_geo_test.csv'.format(DATA_PATH))

In [ ]:
train.head(2)

,point_id,lon,lat,target
0,ommNZCUV,37.590776,55.848630,-0.348157
1,nMe2LHPb,37.784210,55.750271,1.294206


Код ниже получает по широте и долготе адресс. Из него я беру распарсенную часть адресса и сохраняю его в отдельном файле, чтобы не ждать в следующие разы

In [ ]:
"""
geolocator = Nominatim()
locations = []

for i in range(train.shape[0]):
    locations.append(geolocator.reverse(', '.join([str(train.loc[i]['lat']), str(train.loc[i]['lon'])])))

locations_dict = {}
for i in range(len(locations)):
    locations_dict[i] = locations[i].raw['address']

with open('address.txt', 'w') as f:
    json.dump(locations_dict, f, ensure_ascii=False)
"""

In [ ]:
with open('{}/address.txt'.format(DATA_PATH)) as json_file:
    addresses = json.load(json_file)
addresses['0']

{'administrative': 'Северо-Восточный административный округ',
 'city': 'Москва',
 'country_code': 'ru',
 'house_number': '6А',
 'postcode': '127106',
 'region': 'Центральный федеральный округ',
 'road': 'Сигнальный проезд',
 'state': 'Москва',
 'suburb': 'район Отрадное'}

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

data = pd.DataFrame()
data["point_id"] = train["point_id"]
data['point_le'] = le.fit_transform(train["point_id"])
data["lon"] = train["lon"]
data["lat"] = train["lat"]

data['city'] = np.array(['' for i in range(data.shape[0])])
for i in range(data.shape[0]):
    data['city'][i] = addresses[str(i)]['state']
data.head(6)

,point_id,point_le,lon,lat,city
0,ommNZCUV,346,37.590776,55.848630,Москва
1,nMe2LHPb,331,37.784210,55.750271,Москва
2,ZgodVRqB,236,39.635721,47.213330,Ростовская область
3,0t2jNYdz,2,37.704570,55.782020,Москва
4,U27W4QJ7,193,37.643983,55.730188,Москва
5,ci9r9Fr2,259,92.926002,56.065908,Красноярский край


In [ ]:
data['city'].value_counts()

Москва                   160
Санкт-Петербург           83
Самарская область         27
Новосибирская область     26
Татарстан                 25
Свердловская область      22
Ростовская область        21
Нижегородская область     21
Красноярский край         20
Башкортостан              19
Московская область         1
Name: city, dtype: int64

In [ ]:
def get_stations(data, delta):
  api = overpass.API(endpoint="https://lz4.overpass-api.de/api/interpreter")
  response = api.get('node["railway"="station"]({s},{w},{n},{e});out;'.format(s=data['lat'].min()-delta,
                                                                              w=data['lon'].min()-delta,
                                                                              n=data['lat'].max()+delta,
                                                                              e=data['lon'].max()+delta))
  return response

In [ ]:
def get_heat_map(data, city):
  data = data.loc[np.where(data['city'] == city)[0]]
  m = folium.Map(location=[data['lat'].mean(), data['lon'].mean()], zoom_start=10, tiles='cartodbpositron')
  heat_data = [[row['lat'],row['lon']] for index, row in data.iterrows()]
  HeatMap(heat_data, radius=10, gradient={0.4: 'blue', 0.65: '#008080', 1: 'lime'}, blur=5).add_to(m)

  response = get_stations(data, 0.1)
  response = response['features'][:int(len(response['features'])/2)]
  subway_coords = []
  for i in range(len(response)):
    lon = response[i]['geometry']['coordinates'][0]
    lat = response[i]['geometry']['coordinates'][1]
    subway_coords.append([lat, lon])
  HeatMap(subway_coords, radius=5, gradient={0.4: 'red', 0.65: '#8B0000', 1: 'black'}, blur=5).add_to(m)

  return m

get_heat_map(data, 'Москва')

Соберу все станции совершив поиск по городам

In [ ]:
def collect_subway(data, city):
  data = data.loc[np.where(data['city'] == city)[0]]
  response = get_stations(data, 0.1)
  if len(response['features']) == 0:
    return [(np.nan, np.nan, city)]
  
  collection = []
  response = response['features'][:int(len(response['features'])/2)]
  for i in range(len(response)):
    lon = response[i]['geometry']['coordinates'][0]
    lat = response[i]['geometry']['coordinates'][1]
    collection.append((lat, lon, city))
  
  return collection

In [ ]:
subway_data = pd.DataFrame([], columns=['lat', 'lon', 'city'])
ind = 0
for city in data['city'].value_counts().keys():
  collection = collect_subway(data, city)
  for rec in collection:
    subway_data.loc[ind] = rec
    ind += 1

subway_data

,lat,lon,city
0,55.778834,37.653721,Москва
1,55.949653,37.299001,Москва
2,55.726868,37.449885,Москва
3,55.887177,37.661550,Москва
4,55.869625,37.664184,Москва
...,...,...,...
674,55.717956,37.782448,Московская область
675,55.726948,37.753202,Московская область
676,55.764922,37.706704,Московская область
677,55.753643,37.719203,Московская область


In [ ]:
def add_info(data, subway_data, radius=0.02):
  cities = data['city'].value_counts().keys()
  counts = []
  distances = []
  for city in cities:
    print(city)
    data_city = data.loc[np.where(data['city'] == city)[0]].reset_index().drop(columns=['index'])
    subway_city = subway_data.loc[np.where(subway_data['city'] == city)[0]].reset_index().drop(columns=['index'])

    for i in range(data_city.shape[0]):
      buffer = gpd.GeoDataFrame(geometry=[Point(data_city.loc[i]['lat'], data_city.loc[i]['lon'])], 
                                crs={'init': 'epsg:4326'}).buffer(radius)
      count = 0
      dist = []
      for j in range(subway_city.shape[0]):
        count += Point(subway_city.loc[j]['lat'], subway_city.loc[j]['lon']).within(buffer[0])
        dist.append(distance.geodesic((data_city.loc[i]['lat'], data_city.loc[i]['lon']), 
                                           (subway_city.loc[j]['lat'], subway_city.loc[j]['lon'])).m)
      
      counts.append(count)
      distances.append(min(dist) if len(dist) != 0 else 5000.0)
  return counts, distances

попробуй использовать node["place"="city"]["name"="Санкт-Петербург"];out; для нахождения центра города

In [ ]:
counts, distances = add_info(data, subway_data, radius=0.02)

Москва
Санкт-Петербург
Самарская область
Новосибирская область
Татарстан
Свердловская область
Ростовская область
Нижегородская область
Красноярский край
Башкортостан
Московская область


In [ ]:
data['count'] = np.array(counts)
data['min_distance'] = np.array(distances)
data.head(4)

,point_id,point_le,lon,lat,city,count,min_distance
0,ommNZCUV,346,37.590776,55.848630,Москва,4,159.362293
1,nMe2LHPb,331,37.784210,55.750271,Москва,2,176.020610
2,ZgodVRqB,236,39.635721,47.213330,Ростовская область,4,59.381043
3,0t2jNYdz,2,37.704570,55.782020,Москва,10,221.826115


### Fit model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(data[['count']], train[['target']], test_size=0.2, random_state=42, shuffle=True)

In [ ]:
X_train.head()

,count
17,2
66,2
132,6
222,11
31,2


In [ ]:
y_train.head()

,target
17,0.645905
66,0.192094
132,-0.650698
222,-0.261717
31,-1.039679


In [ ]:
model = LinearRegression().fit(X_train, y_train)

In [ ]:
mean_absolute_error(y_valid, model.predict(X_valid))

0.6346442794898969

### Make submission

#### Преобразую данные

In [ ]:
geolocator = Nominatim()
locations = []

for i in range(test.shape[0]):
    locations.append(geolocator.reverse(', '.join([str(test.loc[i]['lat']), str(test.loc[i]['lon'])])))

locations_dict = {}
for i in range(len(locations)):
    locations_dict[i] = locations[i].raw['address']

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

data = pd.DataFrame()
data["point_id"] = test["point_id"]
data['point_le'] = le.fit_transform(test["point_id"])
data["lon"] = test["lon"]
data["lat"] = test["lat"]

data['city'] = np.array(['' for i in range(data.shape[0])])
for i in range(data.shape[0]):
    data['city'][i] = locations_dict[i]['state']
data

,point_id,point_le,lon,lat,city
0,F4lXR1cG,24,37.681242,55.748040,Москва
1,4LJu4GTf,6,60.580910,56.795860,Свердловская область
2,kLuAAN3s,82,37.598614,55.781357,Москва
3,OxQHvaNu,38,37.794051,55.717468,Москва
4,paQsTa1K,91,49.213026,55.748290,Татарстан
...,...,...,...,...,...
102,y8oQuX5v,104,30.353777,60.049792,Санкт-Петербург
103,4nmfqUw0,7,92.928927,56.116262,Красноярский край
104,N9O45mAh,34,93.015993,56.023697,Красноярский край
105,h2InCLKa,73,30.381172,59.871149,Санкт-Петербург


In [ ]:
subway_data = pd.DataFrame([], columns=['lat', 'lon', 'city'])
ind = 0
for city in data['city'].value_counts().keys():
  collection = collect_subway(data, city)
  for rec in collection:
    subway_data.loc[ind] = rec
    ind += 1

subway_data

ServerLoadError: ignored

In [ ]:
counts, distances = add_info(data, subway_data, radius=0.01)

In [ ]:
data['count'] = np.array(counts)
data['min_distance'] = np.array(distances)
data.head(4)

In [ ]:
X = data[['count']]
X.head()

In [ ]:
submission = pd.DataFrame([], columns=['point_id', 'target'])
submission['point_id'] = data['point_id'].values
submission['target'] = model.predict(X)
submission.head()

,point_id,target
0,F4lXR1cG,0.002239
1,4LJu4GTf,-0.021419
2,kLuAAN3s,-0.051633
3,OxQHvaNu,-0.051704
4,paQsTa1K,-0.046585


In [ ]:
submission.to_csv('submission_02.csv', index=False)